# Libraries

In [1]:
import pandas as pd
from datetime import datetime

# Parameters

In [22]:
TAX_RATE = 0.275
METHOD = "HIFO" # ACB, FIFO, LIFO, HIFO
VERBOSE = True

# Functions

In [23]:
def tax_calculator_ACB(selling_units, selling_price, average_purchase_price):
    """
    Calculate the profit and taxes based on selling units, selling price,
    and average purchase price.

    The function computes the total purchase value of the units sold at
    their average purchase price, calculates the profit by subtracting
    the purchase value from the selling value, and applies taxes only
    if the profit is positive.

    Args:
        selling_units (int or float): Number of units sold.
        selling_price (float): Selling price per unit.
        average_purchase_price (float): Average purchase price per unit.

    Returns:
        tuple: A tuple containing:
            - capital_gain (float): The calculated capital gain from the sale.
            - taxes (float): The calculated taxes based on profit and a
              predefined TAX_RATE. If no profit, taxes will be 0.
    """
    # Purchase value that the selling units would have if bought at the same price
    average_purchase_value = selling_units * average_purchase_price

    # Profit: current selling value - average purchase value
    sold_value = selling_units * selling_price
    capital_gain = sold_value - average_purchase_value

    # Calculate taxes only if profit is positive
    taxes = 0
    if capital_gain > 0:
        taxes = capital_gain * TAX_RATE

    return capital_gain, taxes

def calculate_units_to_remove_from_purchase_order(purchase_order_units, units_to_sell_avg, number_of_purchase_orders, counter_order):
  """
  Calculate the number of units to sell and adjust the average units to sell
  per purchase order based on the difference between available purchase order
  units and the average units to sell.

  If the available units in the current purchase order are less than the
  average units to sell, the average is adjusted for the remaining purchase
  orders, and all units from the current order are sold. Otherwise, the
  average units are sold.

  Args:
      purchase_order_units (int or float): The number of units in the current purchase order.
      units_to_sell_avg (float): The average number of units to sell from each purchase order.
      number_of_purchase_orders (int): The total number of purchase orders.
      counter_order (int): The current purchase order index being processed.

  Returns:
      tuple: A tuple containing:
          - units_to_sell (float): The actual number of units to sell from the current order.
          - units_to_sell_avg (float): The updated average units to sell for future orders.
  """

  # Calculate the difference between purchase order units and average units to sell
  units_diff = purchase_order_units - units_to_sell_avg

  if units_diff <0:
    # If not enough units are available, adjust the average units to sell
    n = number_of_purchase_orders  - counter_order # Remaining purchase orders
    units_to_sell_avg += abs(units_diff) / n # Adjust the average
    units_to_sell = purchase_order_units # Sell all available units
  else:
    # If enough units are available, sell the average amount
    units_to_sell = units_to_sell_avg

  return units_to_sell, units_to_sell_avg

def upload_balance(df_balance, df_balance_temp, total_units_to_sell):
    """
    Update the balance of units based on the total units to sell and the temporary balance
    of purchase orders. This function adjusts the units in the original balance DataFrame
    and returns the updated balance along with the total number of units sold.

    Args:
        df_balance (DataFrame): The original balance DataFrame containing current unit counts.
        df_balance_temp (DataFrame): The temporary DataFrame of purchase orders to process.
        total_units_to_sell (int or float): The total number of units to sell from the balance.

    Returns:
        tuple: A tuple containing:
            - df_balance (DataFrame): The updated balance DataFrame after processing.
            - tot_units_sold (int or float): The total number of units sold during the update.
    """

    # Define parameters for balance update
    counter_order = 1  # Count how many purchase orders have been processed
    tot_units_sold = 0  # Keep track of how many units have been removed from balance
    number_of_purchase_orders = df_balance_temp.shape[0]  # Number of purchase orders
    units_to_sell_avg = total_units_to_sell / number_of_purchase_orders  # Average units to remove from each order

    # Update balance
    for j, row_b in df_balance_temp.iterrows():
        # Remove units from purchase orders
        units_to_sell, units_to_sell_avg = calculate_units_to_remove_from_purchase_order(
            purchase_order_units=row_b['Units'],
            units_to_sell_avg=units_to_sell_avg,
            number_of_purchase_orders=number_of_purchase_orders,
            counter_order=counter_order
        )

        # Subtract the units to sell from the balance DataFrame
        df_balance.loc[j, 'Units'] -= units_to_sell

        # Update the counter for the next order
        counter_order += 1

        # Update the total number of units sold
        tot_units_sold += units_to_sell

    # Remove from balance orders which no longer have units
    df_balance = df_balance[df_balance['Units'] > 0].reset_index(drop=True)

    return df_balance, tot_units_sold

def tax_calculator_XYFO(selling_units, df_balance, df_balance_temp):
  """
  Calculate the capital gain and taxes based on selling units, updating the balance of units.

  The function processes the units sold from the current balance, updates the balance DataFrame,
  and calculates the capital gain and taxes. It iterates through the purchase orders, selling
  units from each until the required number of units are sold or no more units are available.

  Args:
      selling_units (int or float): Number of units to sell.
      df_balance (DataFrame): The original balance DataFrame containing current unit counts.
      df_balance_temp (DataFrame): Temporary DataFrame with purchase order details.

  Returns:
      tuple: A tuple containing:
          - capital_gain (float): The calculated capital gain from the sale.
          - taxes (float): The calculated taxes based on capital gain and a predefined TAX_RATE.
          - df_balance (DataFrame): The updated balance DataFrame after processing.
          - tot_units_sold (int or float): The total number of units sold.
  """

  # Initialize variables for tracking progress
  tot_units_sold = 0 # Track how many units have been removed from the balance
  total_units_to_sell = row_s['Units'] # Total units to sell
  units_to_sell = total_units_to_sell  # Units left to sell
  capital_gain = 0 # Initliaze capital gains
  taxes = 0 # Initliaze taxes
  flag = False # Flag to stop the loop when no more units need to be sold

  # Iterate through the temporary balance (purchase orders)
  for j, row_b in df_balance_temp.iterrows():
    # Check if the units in the current purchase order are less than or equal to units to sell
    if row_b['Units'] <= units_to_sell:
      # Sell all units from this purchase order
      df_balance.loc[j, "Units"] = 0 # Set units in the balance to zero
      units_sold = row_b["Units"]    # Number of units sold from this order
      units_to_sell -= units_sold    # Subtract the sold units from units_to_sell
      tot_units_sold +=units_sold    # Add to total units sold
    else:
      # Sell only the required number of units and stop further processing
      units_sold = units_to_sell
      df_balance.loc[j, "Units"] = row_b["Units"] - units_to_sell # Update remaining units
      tot_units_sold += units_sold
      flag = True # Set flag to exit the loop since all required units are sold

    # Calculate capital gain for the units sold from this purchase order
    cg = units_sold*(row_s['Price'] - row_b['Price'])
    capital_gain += cg

    if VERBOSE:
      print(f"Purchased {row_b['Units']} units at {row_b['Price']} and sold them at {row_s['Price']}. Capital gain {round(cg,2)}")

    # Exit the loop once all units are sold
    if flag:
      break

  # Calculate taxes if there's a positive capital gain
  if capital_gain > 0:
    taxes = capital_gain * TAX_RATE

  return capital_gain, taxes, df_balance, tot_units_sold

# Load data

## Purchase

In [ ]:
data = {
    "Date": [
        "01/01/2024", "01/02/2024", "01/03/2024", "01/04/2024", "01/05/2024",
        "01/06/2024", "01/07/2024", "01/08/2024", "01/09/2024", "01/10/2024",
        "01/11/2024", "01/12/2024"
    ],
    "Price": [
        8000, 9000, 6000, 7000, 9000,
        9000, 9000, 12000, 10000, 11000,
        15000, 20000
    ],
    "Units": [0.1] * 12  # Setting all Units to 0.1 for each month
}

# Creating DataFrame
df_purchase = pd.DataFrame(data)
df_purchase['Date'] = pd.to_datetime(df_purchase['Date'], format='%d/%m/%Y')
df_purchase

# Sell

In [ ]:
df_sales_orig = pd.DataFrame({'Date':[datetime(2025,1,31), datetime(2025,3,31)],
                              'Units': [0.33, 0.7],
                              "Price": [7000, 21000]})
df_sales_orig['Value'] = df_sales_orig['Units']*df_sales_orig['Price']
df_sales_orig

# Calculate Taxes

In [ ]:
# Create a copy of:
df_balance = df_purchase.copy() # purchase to keep track of the balance
df_sales = df_sales_orig.copy() # sakes to enrich it with profit/tax infos

if VERBOSE:
  print("Original Balance:")
  print(df_balance)
  print('-------------------------------------------------------------------\n')

# Loop through all sales
sales_infos = []
for i, row_s in df_sales.iterrows():

  # Select purchases until sale date
  df_balance_temp = df_balance[df_balance['Date']<=row_s['Date']]

  # Check if units can be sold:
  if row_s['Units'] <= df_balance_temp['Units'].sum():

    ######################### Average Cost Basis ###############################
    if METHOD == "ACB":

      # Sort filtered balance by value
      df_balance_temp = df_balance_temp.sort_values("Units")

      # Calculate average purchasing price
      average_purchase_price = (df_balance_temp['Units']*df_balance_temp['Price']).sum()/df_balance_temp['Units'].sum()

      # Calculate capital gain and taxes
      capital_gain, taxes = tax_calculator_ACB(selling_units=row_s['Units'],
                                               selling_price=row_s['Price'],
                                               average_purchase_price=average_purchase_price)

      # Store profit/taxes infos in sales dataframe
      sales_infos.append({'Capital Gain':capital_gain,
                          'Taxes':taxes,
                          'Net Profit':capital_gain - taxes,
                          'Average Purchase Price':round(average_purchase_price,2)})

      # Update balance
      df_balance, tot_units_sold = upload_balance(df_balance,
                                                  df_balance_temp,
                                                  row_s['Units'])

      if VERBOSE:
        print(f"Average Purchase Price: {round(average_purchase_price,2)}")
        print(f"Sold {row_s['Units']} units for a value of {row_s['Value']}. The {row_s['Units']} units where purchased at averaged price of {round(average_purchase_price,2)}, which means their average value is {round(row_s['Units']*average_purchase_price, 2)}")
        print(f"Capital Gain: {round(capital_gain,2)} ({round(row_s['Value'], 2)} - {round(row_s['Units']*average_purchase_price,2)})")
        print(f"Taxes: {taxes}\n")

    ############################## FIFO|LIFO|HIFO ##############################
    elif METHOD in ["FIFO", "LIFO", "HIFO"]:

      if METHOD == "FIFO":
        # Sort filtered balance by date
        df_balance_temp = df_balance_temp.sort_values("Date")
      elif METHOD == "LIFO":
        # Sort filtered balance by date in descending order
        df_balance_temp = df_balance_temp.sort_values("Date", ascending=False)
      elif METHOD == "HIFO":
        # Sort filtered balance by value in descending order
        df_balance_temp = df_balance_temp.sort_values("Price", ascending=False)


      # Calculate capital gain, taxes and update balance
      capital_gain, taxes, df_balance, tot_units_sold = tax_calculator_XYFO(selling_units=row_s['Units'],
                                                                            df_balance=df_balance,
                                                                            df_balance_temp=df_balance_temp)

      # Store profit/taxes infos in sales dataframe
      sales_infos.append({'Capital Gain':capital_gain,
                          'Taxes':taxes,
                          'Net Profit':capital_gain - taxes})

      # Remove from balance orders which no longer have units
      df_balance = df_balance[df_balance['Units'] > 0].reset_index(drop=True)

      if VERBOSE:
        print(f"\nSold {row_s['Units']} units for a value of {round(row_s['Value'],2)}.")
        print(f"Capital Gain: {round(capital_gain,2)}")
        print(f"Taxes: {taxes}\n")

    ################################## OTHERS ##################################
    else:
          raise Exception(f"Method not implemented: {METHOD}")

    # Check if sold units match target:
    if round(row_s['Units']) != round(tot_units_sold):
      raise Exception(f"Sold units ({round(tot_units_sold)}) != target ({round(row_s['Units'])})")

    if VERBOSE:
      print("Updated Balance:")
      print(df_balance)
      print('-------------------------------------------------------------------\n')
  else:
    raise Exception(f"Too many units to be sold! {row_s['Units']} > {df_balance_temp['Units'].sum()}")

# Add sales additional infos
df_sales = df_sales.join(pd.DataFrame(sales_infos))

df_sales